<a href="https://colab.research.google.com/github/Ry02024/X_bot/blob/main/notebooks/X_RAG_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 文章生成

In [ ]:
import os
import random
import requests
from requests_oauthlib import OAuth1
import google.generativeai as genai

In [ ]:
from google.colab import userdata
# 環境変数からAPIキーとXの認証情報を取得
GEMINI_API_KEY = userdata.get("Gemini_api")
X_API_KEY = userdata.get("X_API_KEY")
X_API_SECRET = userdata.get("X_API_SECRET")
X_ACCESS_TOKEN = userdata.get("X_ACCESS_TOKEN")
X_ACCESS_TOKEN_SECRET = userdata.get("X_ACCESS_TOKEN_SECRET")

In [ ]:
# 必須環境変数の確認
if not all([GEMINI_API_KEY, X_API_KEY, X_API_SECRET, X_ACCESS_TOKEN, X_ACCESS_TOKEN_SECRET]):
    raise ValueError("必要な環境変数が設定されていません。")

In [ ]:
# Geminiの初期設定
def configure_gemini(api_key):
    genai.configure(api_key=api_key)
    print("Gemini APIの設定が完了しました。")

configure_gemini(GEMINI_API_KEY)

Gemini APIの設定が完了しました。


In [ ]:
# トピックリスト
TOPICS = [
    "食生活の工夫 - 厚めの鶏ハムやチゲ丼など、健康的かつ簡単に作れるレシピ。",
    "時間管理とリフレッシュ方法 - 忙しいスケジュールの中で効率よく休む方法やストレス解消の工夫。",
    "趣味の探索 - 新しい趣味やスキル（例えば絵画、陶芸、音楽など）に挑戦する方法。",
    "心地よい生活空間作り - ミニマリズムや片付けの工夫で、居心地の良い部屋を作るヒント。",
    "運動と健康管理 - 日常生活に取り入れられる軽い運動や健康的な生活習慣。",
    "言語学習の工夫 - 効率的な英語学習法や、実生活に役立つフレーズの習得。",
    "テクノロジーを活用した生活の最適化 - 家事やスケジュール管理に役立つアプリやツールの活用法。",
    "自己成長のための読書 - 日々の生活やキャリアにインスピレーションを与える書籍の選び方。",
    "家族や友人との時間の過ごし方 - 大切な人ともっと充実した時間を過ごすためのアイデア。",
    "季節ごとの楽しみ方 - 季節に合わせた旅行プランや、趣味（花見、紅葉狩り、雪景色の楽しみ方）。"
]

In [ ]:
# トピックをランダムに選択
def select_random_topic():
    return random.choice(TOPICS)

In [ ]:
# 選択されたトピックに基づいて記事を生成
def generate_article(topic):
    prompt = f"""
    以下のトピックについて、100字程度で簡潔かつ具体的に丁寧語（です・ます調）で説明してください。
    トピック: {topic}
    """

    try:
        response = genai.GenerativeModel(model_name="gemini-1.5-pro").generate_content(contents=[prompt])
        generated_text = response.text.strip() if response.text else "記事を生成できませんでした。"
        return generated_text
    except Exception as e:
        raise Exception(f"Gemini APIエラー: {e}")

In [ ]:
# 140字に切り詰める
def trim_to_140_chars(text):
    return text[:140]

In [ ]:
# Xに投稿する
def post_to_x(text):
    auth = OAuth1(X_API_KEY, X_API_SECRET, X_ACCESS_TOKEN, X_ACCESS_TOKEN_SECRET)
    url = "https://api.twitter.com/2/tweets"
    headers = {"Content-Type": "application/json"}
    payload = {"text": text}

    response = requests.post(url, auth=auth, headers=headers, json=payload)

    if response.status_code != 201:
        raise Exception(f"Xへの投稿に失敗しました: {response.status_code} {response.text}")
    print(f"✅ Xに投稿しました: {text}")

In [ ]:
# メイン処理
if __name__ == "__main__":
    try:
        # トピックをランダムに選択
        topic = select_random_topic()
        print(f"選択されたトピック: {topic}")

        # 記事を生成
        article = generate_article(topic)
        print(f"生成された記事: {article}")

        # 140字に切り詰める
        tweet_content = trim_to_140_chars(article)
        print(f"投稿する文章（140字以内）: {tweet_content}")

        # # Xに投稿
        # post_to_x(tweet_content)
    except Exception as e:
        print(f"❌ エラーが発生しました: {e}")

選択されたトピック: テクノロジーを活用した生活の最適化 - 家事やスケジュール管理に役立つアプリやツールの活用法。
生成された記事: 家事やスケジュール管理を効率化するために、様々なアプリやツールが活用できます。例えば、レシピアプリで献立作成の手間を省いたり、スマートスピーカーにリマインダーを設定して予定を管理したりできます。掃除ロボットやスマート家電を活用すれば、家事の負担を軽減可能です。これらのテクノロジーを組み合わせることで、日々の生活をより快適に最適化できます。
投稿する文章（140字以内）: 家事やスケジュール管理を効率化するために、様々なアプリやツールが活用できます。例えば、レシピアプリで献立作成の手間を省いたり、スマートスピーカーにリマインダーを設定して予定を管理したりできます。掃除ロボットやスマート家電を活用すれば、家事の負担を軽減可能です。これらのテクノロジー


# RAG

In [ ]:
!pip install faiss-cpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 35.3 MB/s eta 0:00:00


In [ ]:
import os
import random
import numpy as np
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import faiss

In [ ]:
genai.configure(api_key=GEMINI_API_KEY)  # Gemini APIの設定完了

# サンプルドキュメント（例として修士論文の一部をシミュレーション）
DOCUMENT_TEXT = """
第1章　はじめに
本研究では、XXXXについて検討する。研究の背景として、YYYYが存在し、その解決策としてZZZZが提案されている。本論文では、その有効性を検証するため、様々な実験を行った。

第2章　関連研究
過去の研究では、AAAAやBBBBが報告されている。これらの研究の成果を踏まえ、本研究では新たなアプローチとしてCCCCを採用した。

第3章　実験と考察
実験結果から、提案手法は従来手法に比べて顕著な改善が認められた。これにより、本研究の有効性が示された。
"""

# 生成用のトピックリスト（例）
TOPICS = [
    "研究の背景と意義",
    "関連研究のまとめ",
    "実験結果とその解釈"
]

# テキストをチャンクに分割する関数
def split_text_into_chunks(text: str, chunk_size: int = 150) -> list:
    """
    改行ごとに分割し、chunk_size文字以内の文をまとめるシンプルな例です。
    実際は任意のトークン数や文章単位で分割する方法も検討してください。
    """
    lines = text.split('\n')
    chunks = []
    current_chunk = ""
    for line in lines:
        if not line.strip():
            continue
        # 現在のチャンクに付け加えてもchunk_sizeを超えなければ追加
        if len(current_chunk) + len(line) <= chunk_size:
            current_chunk += line + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = line
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# 埋め込みモデルで各チャンクのエンベディングを生成する関数
def embed_chunks(chunks: list, model: SentenceTransformer) -> np.ndarray:
    embeddings = model.encode(chunks)
    return embeddings

# FAISSインデックスを作成する関数
def build_faiss_index(embeddings: np.ndarray) -> faiss.IndexFlatL2:
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

# トピックに対して関連するチャンクを検索する関数
def search_chunks(query: str, model: SentenceTransformer, index: faiss.IndexFlatL2, chunks: list, top_k: int = 2):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    # FAISSはインデックス番号で返すため、対応するチャンクをリスト化
    retrieved = [chunks[i] for i in indices[0]]
    return retrieved

# Geminiに文脈情報を含むプロンプトで記事生成を依頼する関数
def generate_article_with_context(topic: str, context: str) -> str:
    prompt = f"""以下のトピックと関連情報に基づき、100字程度で簡潔かつ具体的に丁寧語（です・ます調）で説明してください。
トピック: {topic}
関連情報:
{context}
"""
    try:
        # Gemini generative APIにプロンプトを送信
        response = genai.GenerativeModel(model_name="gemini-1.5-pro").generate_content(contents=[prompt])
        generated_text = response.text.strip() if response.text else "記事を生成できませんでした。"
        return generated_text
    except Exception as e:
        return f"Gemini APIエラー: {e}"

In [ ]:
# 1. ドキュメントをチャンクに分割
chunks = split_text_into_chunks(DOCUMENT_TEXT, chunk_size=150)
print("=== Document Chunks ===")
for idx, chunk in enumerate(chunks):
    print(f"{idx}: {chunk}\n")

=== Document Chunks ===
0: 第1章　はじめに 本研究では、XXXXについて検討する。研究の背景として、YYYYが存在し、その解決策としてZZZZが提案されている。本論文では、その有効性を検証するため、様々な実験を行った。 第2章　関連研究

1: 過去の研究では、AAAAやBBBBが報告されている。これらの研究の成果を踏まえ、本研究では新たなアプローチとしてCCCCを採用した。第3章　実験と考察 実験結果から、提案手法は従来手法に比べて顕著な改善が認められた。これにより、本研究の有効性が示された。



In [ ]:
# 2. 埋め込みモデルの初期化とエンベディング生成
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embed_chunks(chunks, model)
print("Embeddings生成完了。")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings生成完了。


In [ ]:
# 3. FAISSインデックスの構築
index = build_faiss_index(embeddings)
print("FAISS index構築完了。")

FAISS index構築完了。


In [ ]:
# 4. ランダムにトピックを選択し、関連チャンクを検索
topic = random.choice(TOPICS)
print("選択されたトピック:", topic)
relevant_chunks = search_chunks(topic, model, index, chunks, top_k=2)
context_text = "\n".join(relevant_chunks)
print("=== 関連するコンテキスト ===")
print(context_text)

選択されたトピック: 研究の背景と意義
=== 関連するコンテキスト ===
過去の研究では、AAAAやBBBBが報告されている。これらの研究の成果を踏まえ、本研究では新たなアプローチとしてCCCCを採用した。第3章　実験と考察 実験結果から、提案手法は従来手法に比べて顕著な改善が認められた。これにより、本研究の有効性が示された。
第1章　はじめに 本研究では、XXXXについて検討する。研究の背景として、YYYYが存在し、その解決策としてZZZZが提案されている。本論文では、その有効性を検証するため、様々な実験を行った。 第2章　関連研究


In [ ]:
# 5. 関連情報を含めた記事生成
article = generate_article_with_context(topic, context_text)
print("=== 生成された記事 ===")
print(article)

=== 生成された記事 ===
過去の研究ではAAAAやBBBBが報告されていますが、YYYYという課題に対しZZZZの有効性検証が十分ではありません。本研究では、新たなアプローチCCCCを採用し、XXXXについて検討しました。第3章の実験結果から、提案手法は従来手法より顕著な改善を示し、ZZZZの有効性、ひいてはYYYY解決への貢献が期待されます。


# RAG2

In [ ]:
# 蓄積された知識からランダムに数件のチャンクを取得する関数
def get_random_context(chunks: list, num_chunks: int = 2) -> str:
    sampled = random.sample(chunks, min(num_chunks, len(chunks)))
    return "\n".join(sampled)

In [ ]:
# RAGの知識を元に適当なツイートを生成する関数
def generate_tweet_with_rag(context: str) -> str:
    prompt = f"""以下の関連情報に基づき、あなたが言いそうなツイートを、丁寧語で100字前後の短文として生成してください。
関連情報:
{context}
"""
    try:
        response = genai.GenerativeModel(model_name="gemini-1.5-pro").generate_content(contents=[prompt])
        generated_text = response.text.strip() if response.text else "記事を生成できませんでした。"
        return generated_text
    except Exception as e:
        return f"Gemini APIエラー: {e}"

In [ ]:
# 4. ランダムな文脈情報を取得（トピック指定はせず、知識ベースからランダムに取得）
context_text = get_random_context(chunks, num_chunks=2)
print("=== 取得されたランダム文脈情報 ===")
print(context_text)

=== 取得されたランダム文脈情報 ===
過去の研究では、AAAAやBBBBが報告されている。これらの研究の成果を踏まえ、本研究では新たなアプローチとしてCCCCを採用した。第3章　実験と考察 実験結果から、提案手法は従来手法に比べて顕著な改善が認められた。これにより、本研究の有効性が示された。
第1章　はじめに 本研究では、XXXXについて検討する。研究の背景として、YYYYが存在し、その解決策としてZZZZが提案されている。本論文では、その有効性を検証するため、様々な実験を行った。 第2章　関連研究


In [ ]:
# 5. 文脈情報を元にGemini APIでツイート文を生成
tweet = generate_tweet_with_rag(context_text)
print("=== 生成されたツイート文 ===")
print(tweet)

=== 生成されたツイート文 ===
過去のAAAA、BBBBといった研究成果を踏まえ、新たなアプローチCCCCを採用したXXXXに関する研究について、論文にまとめました。第3章では実験と考察を行い、提案手法の有効性を実証。従来手法と比べ顕著な改善が見られました。詳細はこちらでご覧ください。(論文へのリンク)#XXXX #YYYY #ZZZZ


# RAG3

In [ ]:
!pip install python-docx -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.9 MB/s eta 0:00:00


In [ ]:
# 必要なライブラリのインストール例（Colab であれば !pip install python-docx sentence-transformers faiss-cpu など）
import os
import numpy as np
from docx import Document
from sentence_transformers import SentenceTransformer
import faiss

In [ ]:
# 1. DOCX からのテキスト抽出
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

In [ ]:
# 2. テキストをチャンクに分割（ここでは簡単な例）
def split_text(text, max_length=300):
    sentences = text.split('\n')
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if not sentence.strip():
            continue
        if len(current_chunk) + len(sentence) <= max_length:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [ ]:
# 3. 埋め込み生成
def compute_embeddings(chunks, model):
    embeddings = model.encode(chunks)
    return np.array(embeddings)

# 4. FAISS インデックスの作成
def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

In [ ]:
# 蓄積された知識からランダムに数件のチャンクを取得する関数
def get_random_context(chunks: list, num_chunks: int = 2) -> str:
    sampled = random.sample(chunks, min(num_chunks, len(chunks)))
    return "\n".join(sampled)

In [ ]:
# RAGの知識を元に適当なツイートを生成する関数
def generate_tweet_with_rag(context: str) -> str:
    prompt = f"""以下の関連情報に基づき、あなたが言いそうなツイートを、丁寧語で100字前後の短文として生成してください。
関連情報:
{context}
"""
    try:
        response = genai.GenerativeModel(model_name="gemini-1.5-pro").generate_content(contents=[prompt])
        generated_text = response.text.strip() if response.text else "記事を生成できませんでした。"
        return generated_text
    except Exception as e:
        return f"Gemini APIエラー: {e}"

In [ ]:
# ※ファイルパスは適宜変更してください
file_path = "/content/161217 Ryo 修士論文 (1).docx"
text = read_docx(file_path)
chunks = split_text(text, max_length=300)
print("分割されたチャンク数:", len(chunks))

分割されたチャンク数: 76


In [ ]:
# SentenceTransformer を利用して埋め込み生成
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = compute_embeddings(chunks, model)

In [ ]:
# FAISS インデックスの構築
index = build_faiss_index(embeddings)
print("FAISS インデックス構築完了。")

FAISS インデックス構築完了。


In [ ]:
# 4. ランダムな文脈情報を取得（トピック指定はせず、知識ベースからランダムに取得）
context_text = get_random_context(chunks, num_chunks=2)
print("=== 取得されたランダム文脈情報 ===")
print(context_text)

=== 取得されたランダム文脈情報 ===
法律と個人の考え方の差を明確にする。先のアンケートで5個の主成分を得た。これらの質問は具体的な行為に対する是非を問うものである。また、外的基準として法律と社会規範を使用し、望ましい回答を作成した。インタビューでは法律と個人の考えの差を明確にするが、その個人の考えもまた法律に影響されているはずである。特に、情報化社会と言われる現代ではパソコンやスマホによる影響も考えられ、その個人が、どこまでを良とし、どこからが問題と考えているかを5個の主成分の観点から考察する。
情報教育研究会（IEC)情報倫理教育研究グループ編（2014）”インターネットの光と影Ver.5”北大路書房 黒須正明（2013）”人間中心設計の基礎”近代科学社 フランク・モス著、千葉敏生訳（2012）”MITメディアラボ魔法のイノベーション・パワー”早川書房 G.ホフステードら著、岩井八郎ら訳（2013）”多文化世界 違いを学び未来への道を探る【原書第3版】”有斐閣 涌井良幸、涌井貞美（2003）”図解でわかる　統計解析用語辞典”日本実業出版社 涌井良幸、涌井貞美（2005）”ピタリとわかる　多変量解析入門”誠文堂新光社


In [ ]:
# 5. 文脈情報を元にGemini APIでツイート文を生成
tweet = generate_tweet_with_rag(context_text)
print("=== 生成されたツイート文 ===")
print(tweet)

=== 生成されたツイート文 ===
法律と個人の倫理観の差異について研究を進めております。アンケート調査(5主成分)とインタビューから、情報化社会における個人の判断基準を探っています。特に、PCやスマホの影響下で、どこまでを「良し」とするのか？文献も参考に考察を深め、後日改めてご報告いたします。 #情報倫理 #倫理観 #法律


# 生成

In [ ]:
# 4. ランダムな文脈情報を取得（トピック指定はせず、知識ベースからランダムに取得）
context_text = get_random_context(chunks, num_chunks=2)
print("=== 取得されたランダム文脈情報 ===")
print(context_text)

=== 取得されたランダム文脈情報 ===
人間を人間足らしめている特性の集合の事で、フロイトの心的構造論のエス(イド)に相当する。無意識的基本的目標または欲求を、簡潔に表した表現。 この項の最後に、基本的欲求から派生する具体的な行動を挙げ、人間性がどのように表現されるかを見る。 　行為・願望の上位概念。普段の生活から特定の願望や行為より導き出した共通する概念。目標より抽象的である。 　欲求が明確に意識され、行為されたモノ。 　普遍的の上位概念。普遍性が基となり、個別の経験より演繹される。 ※基本的欲求から派生する性質 やり方がわかっていることをすること（マズロー(1987)）
選択肢：「重大な問題である、少し問題がある、どちらとも言えない、あまり問題ではない、全く問題ない」 この項では事前に配布する資料を添付する。 「インタビューに関心を持たれた方へ


In [ ]:
# 5. 文脈情報を元にGemini APIでツイート文を生成
tweet = generate_tweet_with_rag(context_text)
print("=== 生成されたツイート文 ===")
print(tweet)

=== 生成されたツイート文 ===
人間の根源的な欲求について考察する資料を添付いたします。フロイトのエスに類似した、行動の源となる「基本的欲求」から、具体的な行動、そして普遍的な性質へと至る過程を解説しています。インタビューにご関心のある方はぜひご覧ください。より深く人間性を理解する一助となれば幸いです。


# まとめ

## プログラム

In [ ]:
!pip install faiss-cpu -q
!pip install python-docx -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
# 必要なライブラリのインストール例（Colab であれば !pip install python-docx sentence-transformers faiss-cpu など）
import os
import random
import numpy as np
from docx import Document
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import faiss

In [ ]:
# 1. DOCX からのテキスト抽出
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

In [ ]:
# 2. テキストをチャンクに分割（ここでは簡単な例）
def split_text(text, max_length=300):
    sentences = text.split('\n')
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if not sentence.strip():
            continue
        if len(current_chunk) + len(sentence) <= max_length:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [ ]:
# 3. 埋め込み生成
def compute_embeddings(chunks, model):
    embeddings = model.encode(chunks)
    return np.array(embeddings)

# 4. FAISS インデックスの作成
def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

In [ ]:
# 蓄積された知識からランダムに数件のチャンクを取得する関数
def get_random_context(chunks: list, num_chunks: int = 2) -> str:
    sampled = random.sample(chunks, min(num_chunks, len(chunks)))
    return "\n".join(sampled)

In [ ]:
# RAGの知識を元に適当なツイートを生成する関数
def generate_tweet_with_rag(context: str) -> str:
    prompt = f"""以下の関連情報に基づき、あなたが言いそうなツイートを、丁寧語で100字前後の短文として生成してください。
                また、自分の知識を既知にすることや、問いかけの表現をやめて下さい。
                インタビューやアンケートについての調査に関する言及もやめて下さい。
                関連情報:
                {context}
                """
    try:
        response = genai.GenerativeModel(model_name="gemini-1.5-pro").generate_content(contents=[prompt])
        generated_text = response.text.strip() if response.text else "記事を生成できませんでした。"
        return generated_text
    except Exception as e:
        return f"Gemini APIエラー: {e}"

## 実行コード

In [ ]:
from google.colab import userdata
# 環境変数からAPIキーとXの認証情報を取得
GEMINI_API_KEY = userdata.get("Gemini_api")
X_API_KEY = userdata.get("X_API_KEY")
X_API_SECRET = userdata.get("X_API_SECRET")
X_ACCESS_TOKEN = userdata.get("X_ACCESS_TOKEN")
X_ACCESS_TOKEN_SECRET = userdata.get("X_ACCESS_TOKEN_SECRET")

In [ ]:
# Geminiの初期設定
def configure_gemini(api_key):
    genai.configure(api_key=api_key)
    print("Gemini APIの設定が完了しました。")

configure_gemini(GEMINI_API_KEY)

Gemini APIの設定が完了しました。


In [ ]:
# ※ファイルパスは適宜変更してください
file_path = "/content/161217 Ryo 修士論文 (1).docx"
text = read_docx(file_path)
chunks = split_text(text, max_length=300)
print("分割されたチャンク数:", len(chunks))

分割されたチャンク数: 76


In [ ]:
# SentenceTransformer を利用して埋め込み生成
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = compute_embeddings(chunks, model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# FAISS インデックスの構築
index = build_faiss_index(embeddings)
print("FAISS インデックス構築完了。")

FAISS インデックス構築完了。


In [ ]:
# 4. ランダムな文脈情報を取得（トピック指定はせず、知識ベースからランダムに取得）
context_text = get_random_context(chunks, num_chunks=2)
print("=== 取得されたランダム文脈情報 ===")
print(context_text)

=== 取得されたランダム文脈情報 ===
法律と個人の考え方の差を明確にする。先のアンケートで5個の主成分を得た。これらの質問は具体的な行為に対する是非を問うものである。また、外的基準として法律と社会規範を使用し、望ましい回答を作成した。インタビューでは法律と個人の考えの差を明確にするが、その個人の考えもまた法律に影響されているはずである。特に、情報化社会と言われる現代ではパソコンやスマホによる影響も考えられ、その個人が、どこまでを良とし、どこからが問題と考えているかを5個の主成分の観点から考察する。
下に上げた言動から、人には守られるべき権利があり、これは自分を尊重することや相手の権利を認めることで問題が発生するのを抑えている。という事が言えそうである。これは「仮説１．権利意識」の「人々は権利を無意識的に判断し、行動している。このため日常の場面ではほとんど権利の衝突はない」について、支持していると考える。 　以下には、権利を侵害しないための具体的な考えや行動についての言動を取り上げた。 3-1. そういう風(ネットで拾う)にやると、要は著作権者にお金がやっぱり入っていかない(権利意識：著作権) 3-2. 法律的に言いか悪いかってのはたぶん悪いんだろうし(権利意識：著作権)


In [ ]:
# 5. 文脈情報を元にGemini APIでツイート文を生成
tweet = generate_tweet_with_rag(context_text)
print("=== 生成されたツイート文 ===")
print(tweet)

=== 生成されたツイート文 ===
法律で認められていても、個人の倫理観で問題を感じるケースがあると感じています。著作権のように、収益が作者に還元されない状況を懸念する声からも、権利意識の大切さが伺えますね。倫理観と法律、社会規範のバランスを改めて考えていきたいです。


問1〜4のような事例も、改めて倫理的に考えてみる必要があると感じました。何かご質問等ございましたら、お気軽にお申し付けください。

の様に、自分の知識を既知にすることや問いかけの表現をやめて下さい。

所有権に関する意識や情報の扱い方について、改めて考える機会をいただきました。私自身も、借りた物やデータの取り扱いには気を配っていきたいと考えています。皆様はいかがでしょうか。